In [8]:
import numpy as np

import pandas as pd

%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import (
    KFold, StratifiedKFold, train_test_split)
from sklearn.feature_selection import RFECV

from sklearn.svm import SVC

import joblib

from libs.container import Container
import dataset

In [2]:
cpu = joblib.cpu_count()

In [5]:
data = dataset.load_scaled()

df = pd.concat([data.b278, data.b261])

cls = {name: idx for idx, name in enumerate(df.tile.unique())}
df["cls"] = df.tile.apply(cls.get)

print(cls)

del data

Reading '/mnt/is0/jbcabral/denoise/dataset/full_scaled.pkl.bz2'
{'b278': 0, 'b261': 1}


In [7]:
BEST_PARAMS = joblib.load("results/hp_selection.pkl.bz2")
SVC_LINEAR_PARAMS = BEST_PARAMS["svc_linear"].best_params_

del BEST_PARAMS

SVC_LINEAR_PARAMS

{'C': 100, 'kernel': 'linear', 'probability': True}

In [12]:
X = df[dataset.FEATURES].values
y = df.cls.values

In [ ]:
%%time
clf = SVC(**SVC_LINEAR_PARAMS)
sel = RFECV(clf, n_jobs=-1, cv=10)
sel.fit(X, y)

In [18]:
joblib.dump(sel, "results/rfecv.pkl.bz2", compress=3)

['results/rfecv.pkl.bz2']